# Data sources cheat sheet

Here's where all the data came from

---

## Notebook admin

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set NumPy and Pandas to print 4 decimal places
np.set_printoptions(precision=4)
pd.set_option("display.precision", 4)

---

# Control groups

### IVT, no treatment, for mRS<=1

Population data from Emberson et al. 2014 (their Figure 2, "Control" column):

> ![](./images/emberson-et-al-2014_figure-2_population-data.png)

We take the rows where NIHSS$\leq$10 to mean patients with nLVO, and NIHSS>10 to mean LVO. Then we can calculate the probability $P$ and odds $O$:

__nLVO:__

$P(\mathrm{mRS}\leq1 \ |\ \mathrm{no\ treatment} \ |\ \mathrm{nLVO}) = \frac{189 + 538}{321 + 1252} = 0.4622 $

$O(\mathrm{mRS}\leq1 \ |\ \mathrm{no\ treatment} \ |\ \mathrm{nLVO}) = \frac{0.46...}{1.0 - 0.46...} = 0.8593 $

__LVO:__

$P(\mathrm{mRS}\leq1 \ |\ \mathrm{no\ treatment} \ |\ \mathrm{LVO}) = \frac{175 + 55 + 8}{808 + 671 + 313} = 0.1328 $

$O(\mathrm{mRS}\leq1 \ |\ \mathrm{no\ treatment} \ |\ \mathrm{LVO}) = \frac{0.13...}{1.0 - 0.13...} = 0.1532 $

---

# Extrapolate odds ratio

Plot of odds ratio with time from Emberson et al. 2014 (their Figure 1):

> ![](./images/emberson-et-al-2014_figure-1_odds-ratio-with-time.png)

> n.b. The y-axis is definitely "odds ratio". The dark blue line was calculated using "log(odds ratio)" and then transformed to "odds ratio" for the graph. 

We take the following two important coordinates from this graph:
    
| Treatment delay (hours) | Odds ratio |    
| --- | --- | 
| 1.0 | $\sim$1.9 |
| 6.3 | 1.0 | 


## Our new calculations

### Method

The following method is explained more fully [here](https://github.com/samuel-book/stroke_outcome/blob/main/extrapolate_odds_ratio.ipynb). 

By converting the two important coordinates to log(odds ratio) and connecting them with a straight line, we can extrapolate back to time zero:

> <img src='./images/extrapolate-emberson-fig1.jpg' width='600'>

The three marked coordinates are:

| Treatment delay (hours) | log(odds ratio) |    
| --- | --- | 
| 0.0 | 0.76296 | 
| 1.0 | 0.64185 |
| 6.3 | 0.00000 | 


When a data point for probability at time 6.3 hours is known, the new log(odds ratio) at time zero can be converted into probability at time zero.


### Usage 

In the "Control Groups" section above, we used data from the Emberson et al. 2014 Figure 2 to find probabilities in the patient population that did not receive treatment. We can feed those probabilities into this straight-line fit to create time-zero probabilities. 

The results:

| Occlusion | $P$(mRS$\leq$1 &#124; time zero) <br> (Extrapolated) | $P$(mRS$\leq$1 &#124; no treatment)  <br> (Emberson et al. 2014, Figure 2) |
| --- | --- | --- |
| nLVO | 0.6483 | 0.4622 |
| LVO | 0.2472 | 0.1328 |

The code to generate these numbers is given at the bottom of [this page](https://github.com/samuel-book/stroke_outcome/blob/main/extrapolate_odds_ratio.ipynb). 

---

# Excess deaths

## MT 

From Goyal et al. 2016 (Table 4, "Mortality" row):

> ![](./images/goyal-et-al-2016_table-4_excess-deaths.png)

In [22]:
prop_fatalICH_MT = (97.0 / 633.0)
prop_fatalICH_noMT = (122.0 / 646.0)
prop_fatalICH_MT_excess = prop_fatalICH_MT - prop_fatalICH_noMT

print(f'% of deaths, given MT:     {100.0 * prop_fatalICH_MT:.4f}')
print(f'% of deaths, no treatment: {100.0 * prop_fatalICH_noMT:.4f}')
print(f'% of excess deaths:        {100.0 * prop_fatalICH_MT_excess:.4f}')

% of deaths, given MT:     15.3239
% of deaths, no treatment: 18.8854
% of excess deaths:        -3.5616


## IVT

From Emberson et al. 2014 (page 5, penultimate paragraph before Discussion):

> ![](./images/emberson-et-al-2014_text_excess-deaths.png)

The 2.7% and 0.4% values given in the text are the proportion of fatal haemorrhages out of the population:

In [19]:
prop_fatalICH_IVT = (91.0 / 3391.0)
prop_fatalICH_noIVT = (13.0 / 3365.0)
prop_fatalICH_IVT_excess = prop_fatalICH_IVT - prop_fatalICH_noIVT

print(f'% of deaths, given IVT:    {100.0 * prop_fatalICH_IVT:.4f}')
print(f'% of deaths, no treatment: {100.0 * prop_fatalICH_noIVT:.4f}')
print(f'% of excess deaths:        {100.0 * prop_fatalICH_IVT_excess:.4f}')

% of deaths, given IVT:    2.6836
% of deaths, no treatment: 0.3863
% of excess deaths:        2.2972


## Our new calculations:

Done stuff to turn IVT into nLVO and LVO separately

[here](https://github.com/samuel-book/stroke_outcome/blob/main/mRS_datasets_full.ipynb). 

---

# Recanalisation

From Hui et al. 2020 (bottom of page 2031, final paragraph of "Secondary Outcome: Recanalisation" section):

> ![](./images/hui-et-al-2020_text_recanalisation-perc.png)

---

## References

de la Ossa Herrero N, Carrera D, Gorchs M, Querol M, Millán M, Gomis M, et al. Design and Validation of a Prehospital Stroke Scale to Predict Large Arterial Occlusion The Rapid Arterial Occlusion Evaluation Scale. Stroke; a journal of cerebral circulation. 2013 Nov 26;45. 

Emberson J, Lees KR, Lyden P, et al. _Effect of treatment delay, age, and stroke severity on the effects of intravenous thrombolysis with alteplase for acute ischaemic stroke: A meta-analysis of individual patient data from randomised trials._ The Lancet 2014;384:1929–35. doi:10.1016/S0140-6736(14)60584-5

Fransen, P., Berkhemer, O., Lingsma, H. et al. Time to Reperfusion and Treatment Effect for Acute Ischemic Stroke: A Randomized Clinical Trial. JAMA Neurol. 2016 Feb 1;73(2):190–6. DOI: 10.1001/jamaneurol.2015.3886

Goyal M, Menon BK, van Zwam WH, et al. _Endovascular thrombectomy after large-vessel ischaemic stroke: a meta-analysis of individual patient data from five randomised trials._ The Lancet 2016;387:1723-1731. doi:10.1016/S0140-6736(16)00163-X

Hui W, Wu C, Zhao W, Sun H, Hao J, Liang H, et al. Efficacy and Safety of Recanalization Therapy for Acute Ischemic Stroke With Large Vessel Occlusion. Stroke. 2020 Jul;51(7):2026–35. 

IST-3 collaborative group, Sandercock P, Wardlaw JM, Lindley RI, Dennis M, Cohen G, et al. The benefits and harms of intravenous thrombolysis with recombinant tissue plasminogen activator within 6 h of acute ischaemic stroke (the third international stroke trial [IST-3]): a randomised controlled trial. Lancet. 2012 379:2352-63.

Lees KR, Bluhmki E, von Kummer R, et al. _Time to treatment with intravenous alteplase and outcome in stroke: an updated pooled analysis of ECASS, ATLANTIS, NINDS, and EPITHET trials_. The Lancet 2010;375:1695-703. doi:10.1016/S0140-6736(10)60491-6

McMeekin P, White P, James MA, Price CI, Flynn D, Ford GA. Estimating the number of UK stroke patients eligible for endovascular thrombectomy. European Stroke Journal. 2017;2:319–26. 

SAMueL-1 data on mRS before stroke (DOI: 10.5281/zenodo.6896710): https://samuel-book.github.io/samuel-1/descriptive_stats/08_prestroke_mrs.html